In [ ]:
import tensorflow as tf
from Recommender_System.utility.decorator import logger


# 创建一个自定义层，用于处理TensorFlow操作
class ReduceSumLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        # 在这里执行reduce_sum操作
        result = tf.reduce_sum(inputs, axis=1)
        # 应用其他必要的操作
        result = tf.where(result < 0., 0., result)
        result = tf.where(result > 1., 1., result)
        result = result[..., tf.newaxis]
        return result


@logger('初始化LFM模型：', ('n_user', 'n_item', 'dim', 'l2'))
def LFM_model(n_user: int, n_item: int, dim=100, l2=1e-6) -> tf.keras.Model:
    l2 = tf.keras.regularizers.l2(l2)
    user_id = tf.keras.Input(shape=(), name='user_id', dtype=tf.int32)
    u = tf.keras.layers.Embedding(n_user, dim, embeddings_regularizer=l2)(user_id)
    item_id = tf.keras.Input(shape=(), name='item_id', dtype=tf.int32)
    i = tf.keras.layers.Embedding(n_item, dim, embeddings_regularizer=l2)(item_id)

    # 使用自定义层
    x = ReduceSumLayer()([u, i])

    return tf.keras.Model(inputs=[user_id, item_id], outputs=x)


if __name__ == '__main__':
    tf.keras.utils.plot_model(LFM_model(1, 1), 'graph.png', show_shapes=True)
